# Lab 4

Disclaimer: The code has been built upon the starter code provided by Dr. Fosler

In [28]:
# import standard PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # TensorBoard support


# import torchvision module to handle image manipulation
import torchvision
import torchvision.transforms as transforms



## Data Preparation

In [29]:
# Use standard FashionMNIST dataset
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

test_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = False,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

In [92]:
train_set.targets

tensor([9, 0, 0,  ..., 3, 0, 5])

In [30]:
train_set.data[:][:][:].shape

torch.Size([60000, 28, 28])

## Linear 2 Layer DNN

In [93]:
# Build the neural network, expand on top of nn.Module
class Linear_Network(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.fc1 = nn.Linear(in_features=28*28,out_features=200)
    self.fc2 = nn.Linear(in_features=200,out_features=10)


  # define forward function
  def forward(self, t):
    # fc 1
    t=t.reshape(-1,28*28)
    t=self.fc1(t)
    t=F.relu(t)

    # fc 2
    t=self.fc2(t)
    # don't need softmax here since we'll use cross-entropy as activation.

    return t

## Baseline CNN model

In [31]:
# Build the neural network, expand on top of nn.Module
class Network(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.fc1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5,5))
    self.fc2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=(5,5))
    self.fc3 = nn.Linear(in_features=4*4*12,out_features=120)
    self.fc4 = nn.Linear(in_features=120,out_features=60)
    self.fc5 = nn.Linear(in_features=60,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))

  # define forward function
  def forward(self, t):
    # fc 1
    t=self.fc1(t)
    t=F.relu(t)
    t = self.mxpool(t)
    # fc 2
    t=self.fc2(t)
    t=F.relu(t)
    t = self.mxpool(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    
    # don't need softmax here since we'll use cross-entropy as activation.

    return t

## Accuracy calculator

In [32]:
def get_accuracy(model,dataloader):
  count=0
  correct=0

  model.eval()
  with torch.no_grad():
    for batch in dataloader:
      images = batch[0]
      labels = batch[1]
      preds=model(images)
      batch_correct=preds.argmax(dim=1).eq(labels).sum().item()
      batch_count=len(batch[0])
      count+=batch_count
      correct+=batch_correct
  model.train()
  return correct/count


## Evaluating the 2 Layer Linear DNN model

In [ ]:
def train_test_network(network, epochs):
    # set the network to training mode
  loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
  optimizer = optim.Adam(network.parameters(), lr=lr)
  network.train()
  for epoch in range(epochs):
    for batch in loader:
      images = batch[0]
      labels = batch[1]
      preds = network(images)
      loss = F.cross_entropy(preds, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print('Epoch {0}: train set accuracy {1}'.format(epoch,get_accuracy(network,loader)))

  test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
  print('Epoch {0}: test set accuracy {1}'.format(epoch,get_accuracy(network,test_loader)))

In [ ]:
network = Linear_Network()
train_test_network(network, epochs=10)

Epoch 0: train set accuracy 0.7971166666666667
Epoch 1: train set accuracy 0.8341
Epoch 2: train set accuracy 0.84615
Epoch 3: train set accuracy 0.8538666666666667
Epoch 4: train set accuracy 0.8590666666666666
Epoch 5: train set accuracy 0.8640833333333333
Epoch 6: train set accuracy 0.8677
Epoch 7: train set accuracy 0.8714
Epoch 8: train set accuracy 0.8743666666666666
Epoch 9: train set accuracy 0.8775666666666667
Epoch 9: test set accuracy 0.8601


## Evaluating the Baseline CNN model

In [33]:
lr=0.001
batch_size=1000
shuffle=True
epochs=10

network = Network()
loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

# set the network to training mode
network.train()
for epoch in range(epochs):
  for batch in loader:
    images = batch[0]
    labels = batch[1]
    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch,get_accuracy(network,loader)))

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch,get_accuracy(network,test_loader)))




Epoch 0: train set accuracy 0.6918666666666666
Epoch 1: train set accuracy 0.7322666666666666
Epoch 2: train set accuracy 0.75585
Epoch 3: train set accuracy 0.7796333333333333
Epoch 4: train set accuracy 0.7984
Epoch 5: train set accuracy 0.81245
Epoch 6: train set accuracy 0.82285
Epoch 7: train set accuracy 0.83085
Epoch 8: train set accuracy 0.8382333333333334
Epoch 9: train set accuracy 0.8439
Epoch 9: test set accuracy 0.8342


## Experiments with different Models

1. Introduce zero padding and increase the number of feature maps

In [34]:
# Build the neural network, expand on top of nn.Module
class Network1(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
    self.fc3 = nn.Linear(in_features=6*6*64,out_features=120)
    self.fc4 = nn.Linear(in_features=120,out_features=60)
    self.fc5 = nn.Linear(in_features=60,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))

  # define forward function
  def forward(self, t):
    # fc 1
    t=self.conv1(t)
    t=F.relu(t)
    t = self.mxpool(t)
    # fc 2
    t=self.conv2(t)
    t=F.relu(t)
    t = self.mxpool(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    

    return t

In [35]:
lr=0.001
batch_size=1000
shuffle=True
epochs=10

network = Network1()
loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

# set the network to training mode
network.train()
for epoch in range(epochs):
  for batch in loader:
    images = batch[0]
    labels = batch[1]
    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch,get_accuracy(network,loader)))

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch,get_accuracy(network,test_loader)))

Epoch 0: train set accuracy 0.7525666666666667
Epoch 1: train set accuracy 0.8144166666666667
Epoch 2: train set accuracy 0.84085
Epoch 3: train set accuracy 0.8593833333333334
Epoch 4: train set accuracy 0.8655
Epoch 5: train set accuracy 0.8730833333333333
Epoch 6: train set accuracy 0.8812666666666666
Epoch 7: train set accuracy 0.8865166666666666
Epoch 8: train set accuracy 0.8904
Epoch 9: train set accuracy 0.8940333333333333
Epoch 9: test set accuracy 0.8846


2. Introduce dropout layer

In [36]:
# Build the neural network, expand on top of nn.Module
class Network2(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
    self.fc3 = nn.Linear(in_features=6*6*64,out_features=120)
    self.fc4 = nn.Linear(in_features=120,out_features=60)
    self.fc5 = nn.Linear(in_features=60,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))
    self.drop = nn.Dropout2d(0.25)

  # define forward function
  def forward(self, t):
    # fc 1
    t=self.conv1(t)
    t=F.relu(t)
    t = self.mxpool(t)
    t = self.drop(t)
    # fc 2
    t=self.conv2(t)
    t=F.relu(t)
    t = self.mxpool(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.drop(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    

    return t

In [37]:
network = Network2()
loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

# set the network to training mode
network.train()
for epoch in range(epochs):
  for batch in loader:
    images = batch[0]
    labels = batch[1]
    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch,get_accuracy(network,loader)))

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch,get_accuracy(network,test_loader)))

Epoch 0: train set accuracy 0.7626666666666667
Epoch 1: train set accuracy 0.8218833333333333
Epoch 2: train set accuracy 0.8464666666666667
Epoch 3: train set accuracy 0.8650833333333333
Epoch 4: train set accuracy 0.8725
Epoch 5: train set accuracy 0.8824333333333333
Epoch 6: train set accuracy 0.8877666666666667
Epoch 7: train set accuracy 0.8896666666666667
Epoch 8: train set accuracy 0.89715
Epoch 9: train set accuracy 0.8991833333333333
Epoch 9: test set accuracy 0.8899


This model also performs good as we see almost the same train and test set accuracy

3. Increase the size of the linear layer

In [46]:
# Build the neural network, expand on top of nn.Module
class Network3(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
    self.fc3 = nn.Linear(in_features=6*6*64,out_features=600)
    self.fc4 = nn.Linear(in_features=600,out_features=120)
    self.fc5 = nn.Linear(in_features=120,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))
    self.drop = nn.Dropout2d(0.25)

  # define forward function
  def forward(self, t):
    # fc 1
    t=self.conv1(t)
    t=F.relu(t)
    t = self.mxpool(t)
    t = self.drop(t)
    # fc 2
    t=self.conv2(t)
    t=F.relu(t)
    t = self.mxpool(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.drop(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    

    return t

In [47]:
network = Network3()
train_test_network(network)

Epoch 0: train set accuracy 0.79345
Epoch 1: train set accuracy 0.8428333333333333
Epoch 2: train set accuracy 0.8667166666666667
Epoch 3: train set accuracy 0.8826166666666667
Epoch 4: train set accuracy 0.89135
Epoch 5: train set accuracy 0.89755
Epoch 6: train set accuracy 0.9056666666666666
Epoch 7: train set accuracy 0.9067333333333333
Epoch 8: train set accuracy 0.9129166666666667
Epoch 9: train set accuracy 0.9158333333333334
Epoch 9: test set accuracy 0.899


4. Batch normalization before Layer 2 and Layer 3

In [100]:
# Build the neural network, expand on top of nn.Module
class Network4(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
    self.fc3 = nn.Linear(in_features=6*6*64,out_features=600)
    self.fc4 = nn.Linear(in_features=600,out_features=120)
    self.fc5 = nn.Linear(in_features=120,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))
    self.drop = nn.Dropout2d(0.25)
    self.batchnorm1 = nn.BatchNorm2d(32)
    self.batchnorm2 = nn.BatchNorm2d(64)


  # define forward function
  def forward(self, t):
    # conv1 1
    t=self.conv1(t)
    t=F.relu(t)
    t = self.mxpool(t)
    t = self.batchnorm1(t)

    # conv 2
    t=self.conv2(t)
    t=F.relu(t)
    t = self.mxpool(t)
    t = self.batchnorm2(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.drop(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    

    return t

In [101]:
network = Network4()
train_test_network(network, epochs=10)

Epoch 0: train set accuracy 0.89255
Epoch 1: train set accuracy 0.9190666666666667
Epoch 2: train set accuracy 0.9437333333333333
Epoch 3: train set accuracy 0.9544
Epoch 4: train set accuracy 0.9582666666666667
Epoch 5: train set accuracy 0.9669833333333333
Epoch 6: train set accuracy 0.9752166666666666
Epoch 7: train set accuracy 0.9798
Epoch 8: train set accuracy 0.9812166666666666
Epoch 9: train set accuracy 0.9819833333333333
Epoch 9: test set accuracy 0.9128


As can be seen in the results there is overfitting

4.1 Batch normalization immediatly after the convolution layer

In [102]:
# Build the neural network, expand on top of nn.Module
class Network4_1(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
    self.fc3 = nn.Linear(in_features=6*6*64,out_features=600)
    self.fc4 = nn.Linear(in_features=600,out_features=120)
    self.fc5 = nn.Linear(in_features=120,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))
    self.drop = nn.Dropout2d(0.25)
    self.batchnorm1 = nn.BatchNorm2d(32)
    self.batchnorm2 = nn.BatchNorm2d(64)


  # define forward function
  def forward(self, t):
    # conv1 1
    t=self.conv1(t)
    t = self.batchnorm1(t)
    t=F.relu(t)
    t = self.mxpool(t)

    # conv 2
    t=self.conv2(t)
    t = self.batchnorm2(t)

    t=F.relu(t)
    t = self.mxpool(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.drop(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    

    return t

In [103]:
network = Network4_1()
train_test_network(network, epochs=10)

Epoch 0: train set accuracy 0.8748833333333333
Epoch 1: train set accuracy 0.9016833333333333
Epoch 2: train set accuracy 0.9124833333333333
Epoch 3: train set accuracy 0.9234333333333333
Epoch 4: train set accuracy 0.9285666666666667
Epoch 5: train set accuracy 0.9366166666666667
Epoch 6: train set accuracy 0.9332833333333334
Epoch 7: train set accuracy 0.9205666666666666
Epoch 8: train set accuracy 0.9479833333333333
Epoch 9: train set accuracy 0.9549
Epoch 9: test set accuracy 0.9186


5. Increasing the number of epochs to 20

In [54]:
# Build the neural network, expand on top of nn.Module
class Network5(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
    self.fc3 = nn.Linear(in_features=6*6*64,out_features=600)
    self.fc4 = nn.Linear(in_features=600,out_features=120)
    self.fc5 = nn.Linear(in_features=120,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))
    self.drop = nn.Dropout2d(0.25)
    self.batchnorm1 = nn.BatchNorm2d(32)
    self.batchnorm2 = nn.BatchNorm2d(64)


  # define forward function
  def forward(self, t):
    # conv1 1
    t=self.conv1(t)
    # t = self.batchnorm1(t)
    t=F.relu(t)
    t = self.mxpool(t)
    t = self.drop(t)
    # conv 2
    t=self.conv2(t)
    # t = self.batchnorm2(t)
    t=F.relu(t)
    t = self.mxpool(t)
    t = self.drop(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.drop(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    

    return t

In [57]:
network = Network5()
train_test_network(network, epochs=20)

Epoch 0: train set accuracy 0.7832666666666667
Epoch 1: train set accuracy 0.84505
Epoch 2: train set accuracy 0.8646333333333334
Epoch 3: train set accuracy 0.8793166666666666
Epoch 4: train set accuracy 0.8865333333333333
Epoch 5: train set accuracy 0.8944833333333333
Epoch 6: train set accuracy 0.9029
Epoch 7: train set accuracy 0.9061166666666667
Epoch 8: train set accuracy 0.9114833333333333
Epoch 9: train set accuracy 0.9166166666666666
Epoch 10: train set accuracy 0.9188666666666667
Epoch 11: train set accuracy 0.9218166666666666
Epoch 12: train set accuracy 0.9255166666666667
Epoch 13: train set accuracy 0.9300166666666667
Epoch 14: train set accuracy 0.9325
Epoch 15: train set accuracy 0.9344833333333333
Epoch 16: train set accuracy 0.9376
Epoch 17: train set accuracy 0.9385
Epoch 18: train set accuracy 0.9399166666666666
Epoch 19: train set accuracy 0.9423
Epoch 19: test set accuracy 0.9179


6. Increasing the kernel size of the baseline model

In [98]:
# Build the neural network, expand on top of nn.Module
class Network6(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.fc1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(5,5))
    self.fc2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=(5,5))
    self.fc3 = nn.Linear(in_features=4*4*64,out_features=120)
    self.fc4 = nn.Linear(in_features=120,out_features=60)
    self.fc5 = nn.Linear(in_features=60,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))

  # define forward function
  def forward(self, t):
    # fc 1
    t=self.fc1(t)
    t=F.relu(t)
    t = self.mxpool(t)
    # fc 2
    t=self.fc2(t)
    t=F.relu(t)
    t = self.mxpool(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    
    # don't need softmax here since we'll use cross-entropy as activation.

    return t

In [99]:
network = Network6()
train_test_network(network, epochs=10)

Epoch 0: train set accuracy 0.7303833333333334
Epoch 1: train set accuracy 0.7711333333333333
Epoch 2: train set accuracy 0.80215
Epoch 3: train set accuracy 0.8206666666666667
Epoch 4: train set accuracy 0.8393166666666667
Epoch 5: train set accuracy 0.8504333333333334
Epoch 6: train set accuracy 0.85875
Epoch 7: train set accuracy 0.86375
Epoch 8: train set accuracy 0.8726166666666667
Epoch 9: train set accuracy 0.8768666666666667
Epoch 9: test set accuracy 0.8682


## Variation 1 Model 

In [ ]:
# Build the neural network, expand on top of nn.Module
class Network3(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
    self.fc3 = nn.Linear(in_features=6*6*64,out_features=600)
    self.fc4 = nn.Linear(in_features=600,out_features=120)
    self.fc5 = nn.Linear(in_features=120,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))
    self.drop = nn.Dropout2d(0.25)

  # define forward function
  def forward(self, t):
    # fc 1
    t=self.conv1(t)
    t=F.relu(t)
    t = self.mxpool(t)
    t = self.drop(t)
    # fc 2
    t=self.conv2(t)
    t=F.relu(t)
    t = self.mxpool(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.drop(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    

    return t

In [ ]:
network = Network3()
train_test_network(network)

Epoch 0: train set accuracy 0.79345
Epoch 1: train set accuracy 0.8428333333333333
Epoch 2: train set accuracy 0.8667166666666667
Epoch 3: train set accuracy 0.8826166666666667
Epoch 4: train set accuracy 0.89135
Epoch 5: train set accuracy 0.89755
Epoch 6: train set accuracy 0.9056666666666666
Epoch 7: train set accuracy 0.9067333333333333
Epoch 8: train set accuracy 0.9129166666666667
Epoch 9: train set accuracy 0.9158333333333334
Epoch 9: test set accuracy 0.899


## Variation 2 model

In [ ]:
# Build the neural network, expand on top of nn.Module
class Network4_1(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
    self.fc3 = nn.Linear(in_features=6*6*64,out_features=600)
    self.fc4 = nn.Linear(in_features=600,out_features=120)
    self.fc5 = nn.Linear(in_features=120,out_features=10)
    self.mxpool = nn.MaxPool2d(kernel_size=(2,2),stride = (2,2))
    self.drop = nn.Dropout2d(0.25)
    self.batchnorm1 = nn.BatchNorm2d(32)
    self.batchnorm2 = nn.BatchNorm2d(64)


  # define forward function
  def forward(self, t):
    # conv1 1
    t=self.conv1(t)
    t = self.batchnorm1(t)
    t=F.relu(t)
    t = self.mxpool(t)

    # conv 2
    t=self.conv2(t)
    t = self.batchnorm2(t)

    t=F.relu(t)
    t = self.mxpool(t)

    t = torch.flatten(t,1)
    t = self.fc3(t)
    t = F.relu(t)
    t = self.drop(t)
    t = self.fc4(t)
    t = F.relu(t)
    t = self.fc5(t)
    # t = F.log_softmax(t, dim=1)    

    return t

In [ ]:
network = Network4_1()
train_test_network(network, epochs=10)

Epoch 0: train set accuracy 0.8748833333333333
Epoch 1: train set accuracy 0.9016833333333333
Epoch 2: train set accuracy 0.9124833333333333
Epoch 3: train set accuracy 0.9234333333333333
Epoch 4: train set accuracy 0.9285666666666667
Epoch 5: train set accuracy 0.9366166666666667
Epoch 6: train set accuracy 0.9332833333333334
Epoch 7: train set accuracy 0.9205666666666666
Epoch 8: train set accuracy 0.9479833333333333
Epoch 9: train set accuracy 0.9549
Epoch 9: test set accuracy 0.9186
